## Sequence hacking, hashing and slicing


In [2]:
# vecter 扩展

from array import array
import math
import reprlib

class Vecter:
    
    typecode = 'd'
    def __init__(self, components):
        self._components = array(self.typecode, components)
    
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vecter({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)
    
    

### summary

- 由于 \_\_iter\_\_ 的作用，在 eq 比较的时候可以直接用 tuple(self) 表示 (3, 4) 这样的结构
- reprlib.repr 的作用是防止调试信息过长
- list(ob), float(ob), tuple(ob) 等其实是一个确认对象类型的过程，这样就省去了自己写 assert/isinstance 等代码


In [4]:
Vecter([3, 4])

Vecter([3.0, 4.0])

In [5]:
Vecter(range(10))

Vecter([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [7]:
Vecter(str(range(10))) # 保证传入的值为数字

TypeError: cannot use a str to initialize an array with typecode 'd'

In [8]:
ｖ = Vecter(range(10))
v._components

array('d', [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0])

###  a sliceable sequence


In [9]:
# 继承　Ｖecter ，扩展为可索引

class SVecter(Vecter):
    
    def __len__(self):
        
        return len(self._components)
    
    def __getitem__(self, idx):
        
        return self._components[idx]
    
    

In [10]:
v1 = SVecter(range(5))
v1

Vecter([0.0, 1.0, 2.0, 3.0, 4.0])

In [11]:
len(v1)

5

In [13]:
print(v1[:])
print(v1[0])
print(v1[-1])

array('d', [0.0, 1.0, 2.0, 3.0, 4.0])
0.0
4.0


### How slicing works

In [14]:
class Myseq:
    
    def __getitem__(self, idx):
        return idx

In [15]:
m = Myseq()

In [16]:
# 索引
m[1]

1

In [17]:
# 切片
m[1:5:1]

slice(1, 5, 1)

In [18]:
m[1:]

slice(1, None, None)

In [19]:
m[::-1]

slice(None, None, -1)

In [20]:
# 切片+索引
m[1:5:1, 2]

(slice(1, 5, 1), 2)

In [21]:
# 切片＋切片

m[1:9:3, 2:10:6]

(slice(1, 9, 3), slice(2, 10, 6))

In [22]:
# 结合 vecter 

sv = SVecter(range(20))

In [23]:
print(sv[1:10:3, 11])

TypeError: array indices must be integers

In [24]:
print(sv[1:10:3])

array('d', [1.0, 4.0, 7.0])


In [25]:
print(sv[1:10:3, 2:15:6])

TypeError: array indices must be integers

In [26]:
# 直接使用 list 看看

l = list(range(100))
print(l[1:20:6, 35])

TypeError: list indices must be integers or slices, not tuple

In [27]:
# 综上，　切片＋切片/索引　没有太多卵用

### A slice-aware __getitem__


In [34]:
# 改良版

import numbers

class AVecter(Vecter):
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, idx):
        cls = type(self)
        print(cls(range(5)))
        if isinstance(idx, slice):
            return cls(self._components[idx])
        elif isinstance(idx, numbers.Integral):
            return self._components[idx]
        
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
            
        

In [35]:
av = AVecter(range(10))
av[-1]

(0.0, 1.0, 2.0, 3.0, 4.0)


9.0

In [31]:
av[1:4]

Vecter([1.0, 2.0, 3.0])

In [36]:
av['sss']

(0.0, 1.0, 2.0, 3.0, 4.0)


TypeError: AVecter indices must be integers

### dynamic attribute access


In [43]:
# 动态属性指定

class GVecter(Vecter):
    
    shortcut_name = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_name.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
            
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    #  __setattr__ 防止错误指定
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_name:
                error = 'readonly attribute {attr_name!r}'
                
            elif name.islower():
                error = "can't set attr 'a' to 'z' in {cls_name!r}"
                
            else: 
                error = ''
                
            if error:
                msg = error.format(cls_name = cls.__name__, attr_name = name)
                raise AttributeError(msg)
        
        super().__setattr__(name, value)
                

In [44]:
gv = GVecter(range(5))
gv.x

0.0

In [41]:
gv.y

1.0

In [42]:
gv.a

AttributeError: 'GVecter' object has no attribute 'a'

In [45]:
gv.z = 1

AttributeError: readonly attribute 'z'

In [46]:
gv.Q = 10

In [47]:
gv

Vecter([0.0, 1.0, 2.0, 3.0, 4.0])

###  hashing and a faster 

In [50]:
# hash

import functools
import operator

class HVecter(Vecter):
    
    typecode = 'd'
    def __eq__(self, other):
#         return tuple(self) == tuple(other)
        return len(self) == len(other) and all(a==b for a,b in zip(self, other))
    
    def __hash__(self):
#         hashes = (hash(x) for x in self._components)
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    

In [49]:
hv = HVecter(range(5))
hash(hv)

4

In [51]:
hv2 = HVecter(range(5))

hv == hv2

True